## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

# To add time delays to the code
import time

In [2]:
# 1. Import the WeatherPy_database.csv file. 
# keep_default_na --> to prevent pandas from converting country code NA (Namibia) to NaN
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv", keep_default_na=False)

In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Boa Vista,BR,2.8197,-60.6733,78.78,83,20,4.61,few clouds
1,1,Hermanus,ZA,-34.4187,19.2345,52.90,75,0,11.18,clear sky
2,2,Vao,NC,-22.6667,167.4833,73.45,71,95,3.22,overcast clouds
3,3,Kirakira,SB,-10.4544,161.9205,83.05,72,100,5.97,overcast clouds
4,4,Oktyabrskoye,RU,43.0645,44.7417,57.34,94,60,2.37,broken clouds
6,6,Hobart,AU,-42.8794,147.3294,58.05,60,75,7.00,moderate rain
7,7,Wenling,CN,28.3669,121.3600,80.91,91,100,1.95,light rain
8,8,Ribeira Grande,PT,38.5167,-28.7000,63.72,87,100,8.14,overcast clouds
9,9,Deming,US,32.2687,-107.7586,80.49,14,1,5.75,clear sky
10,10,Chuy,UY,-33.6971,-53.4616,62.91,96,98,6.20,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df
clean_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Boa Vista,BR,2.8197,-60.6733,78.78,83,20,4.61,few clouds
1,1,Hermanus,ZA,-34.4187,19.2345,52.90,75,0,11.18,clear sky
2,2,Vao,NC,-22.6667,167.4833,73.45,71,95,3.22,overcast clouds
3,3,Kirakira,SB,-10.4544,161.9205,83.05,72,100,5.97,overcast clouds
4,4,Oktyabrskoye,RU,43.0645,44.7417,57.34,94,60,2.37,broken clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Boa Vista,BR,78.78,few clouds,2.8197,-60.6733,
1,Hermanus,ZA,52.90,clear sky,-34.4187,19.2345,
2,Vao,NC,73.45,overcast clouds,-22.6667,167.4833,
3,Kirakira,SB,83.05,overcast clouds,-10.4544,161.9205,
4,Oktyabrskoye,RU,57.34,broken clouds,43.0645,44.7417,
6,Hobart,AU,58.05,moderate rain,-42.8794,147.3294,
7,Wenling,CN,80.91,light rain,28.3669,121.3600,
8,Ribeira Grande,PT,63.72,overcast clouds,38.5167,-28.7000,
9,Deming,US,80.49,clear sky,32.2687,-107.7586,
10,Chuy,UY,62.91,light rain,-33.6971,-53.4616,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # Group cities in sets of 50 to add time delay. This is to avoid api key blocking
    if (index % 50 == 0 and index >= 50):
        time.sleep(15)

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Boa Vista,BR,78.78,few clouds,2.8197,-60.6733,Aipana Plaza Hotel
1,Hermanus,ZA,52.90,clear sky,-34.4187,19.2345,Misty Waves Boutique Hotel
2,Vao,NC,73.45,overcast clouds,-22.6667,167.4833,Hotel Kou-Bugny
3,Kirakira,SB,83.05,overcast clouds,-10.4544,161.9205,SanBiz Lodge
4,Oktyabrskoye,RU,57.34,broken clouds,43.0645,44.7417,Prolesok Mini-hotel
6,Hobart,AU,58.05,moderate rain,-42.8794,147.3294,Mantra on Collins Hobart
7,Wenling,CN,80.91,light rain,28.3669,121.3600,Hanting Express Wenling Wanchang North Road
8,Ribeira Grande,PT,63.72,overcast clouds,38.5167,-28.7000,Quinta da Meia Eira
9,Deming,US,80.49,clear sky,32.2687,-107.7586,Best Western Deming Southwest Inn
10,Chuy,UY,62.91,light rain,-33.6971,-53.4616,Nuevo Hotel Plaza


In [11]:
# 7. Drop the rows where there is no Hotel Name.
print(hotel_df.isnull().sum())
clean_hotel_df = hotel_df

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64


In [12]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Boa Vista,BR,78.78,few clouds,2.8197,-60.6733,Aipana Plaza Hotel
1,Hermanus,ZA,52.90,clear sky,-34.4187,19.2345,Misty Waves Boutique Hotel
2,Vao,NC,73.45,overcast clouds,-22.6667,167.4833,Hotel Kou-Bugny
3,Kirakira,SB,83.05,overcast clouds,-10.4544,161.9205,SanBiz Lodge
4,Oktyabrskoye,RU,57.34,broken clouds,43.0645,44.7417,Prolesok Mini-hotel
6,Hobart,AU,58.05,moderate rain,-42.8794,147.3294,Mantra on Collins Hobart
7,Wenling,CN,80.91,light rain,28.3669,121.3600,Hanting Express Wenling Wanchang North Road
8,Ribeira Grande,PT,63.72,overcast clouds,38.5167,-28.7000,Quinta da Meia Eira
9,Deming,US,80.49,clear sky,32.2687,-107.7586,Best Western Deming Southwest Inn
10,Chuy,UY,62.91,light rain,-33.6971,-53.4616,Nuevo Hotel Plaza


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} and </dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))